[ja]: #
# Chapter 3. エコーステートネットワーク基礎

[en]: #
# Chapter 3. Basics of Echo State Network

[ja]: #
この章では、リザバー計算 (Reservoir Computing; RC)の文脈で頻繁に使用されるエコーステートネットワーク (Echo State Network; ESN)の基礎と標準的な設定を学びます。

[en]: #
In this chapter, we will study the standard setting and basic implementation of echo state networks (ESNs), which are frequently used in the context of reservoir computing (RC).

[ja]: #
## 前書き

[en]: #
## Introduction

[ja]: #
ESNは、H. Jaeger<sup>[1]</sup>(2001) によって提案されたリカレントニューラルネットワーク (Recurrent Neural Network; RNN)の一種です。
RNNは深層学習で一般的に用いられるフィードフォワードネットワークとは異なり、その再帰結合によって特徴づけられます。
またRCでは一般的なRNN (Long-Short-Term Memory; LSTMやGated Recurrent Unit; GRU等)とは異なり、その再帰結合は固定化されたまま、リードアウト層 (readout layer)と呼ばれる外部のパラメータのみが調整されます。
このような設定で使用されるRNNはリザバー (reservoir)と呼ばれ、RCでは最も単純なセットアップとしてESNがリザバーとして用いられます。

[en]: #

The ESN is a type of recurrent neural network (RNN) proposed by H. Jaeger<sup>[1]</sup> (2001).
Unlike feedforward networks commonly used in deep learning, RNNs are characterized by their recurrent connections.
Additionally, unlike typical RNNs (e.g., long short-term memory or gated recurrent unit), the recurrent connections in ESNs remain fixed, and only the external readout layer parameters are trained.
The RNN used in this configuration is referred to as a **reservoir**, and an ESN is primarily used in RC studies.

[ja]: #
RCはリザバーを用いて様々な時系列タスクを解くフレームワークで、教師あり時系列学習に分類されます。
通常時系列は、毎ステップ入力としてリザバーの内部状態に与えられ、再帰結合と非線形関数に従ってその内部状態が時間発展します。
またRCでは先述のとおり、再帰結合の調整を伴わず、外部のリードアウト層のみの学習だけで多様な非線形な入出力処理を実現します。
ここで重要なポイントとして、リザバーの内部状態の高次元性と非線形性が挙げられます。
一般にリザバーは高次元であるほど高い情報処理能力を有しますが、それは履歴として入力の情報を蓄積できるだけの(記憶)容量を、高次元な設定では確保しやすいからです。
また非線形関数をその活性化関数として有すると、非線形変換が施された多様な入力の情報を保持します。
他にもESNを用いたRCについて一般に以下の特長が挙げられます。
- 非線形な活性化関数を使用 (この点はRNNと同様)
- 通常再帰結合のパラメータをランダムに設定し**固定して**使用
- **外側のリードアウト層のみを学習**するため誤差逆伝搬法等と比して低計算コスト
- 同じリザバーを再利用しながら複数のリードアウト層により**マルチタスキング** (Multitasking; 文脈によってはMultiplexingとも呼ばれる)が容易に可能

[en]: #
RC is a framework that uses reservoirs to solve various time-series tasks and is a supervised learning method for time-series data.
Normally, a time series is fed into the reservoir at each step and the internal state evolves via the recurrent connections and nonlinear activations.
RC can realize many nonlinear input-output mappings by training only the external readout without changing the recurrent connections.

An important feature is the high dimensionality and nonlinearity of the reservoir.
In general, it can store a greater amount of input information as history in a higher-dimensional setting.
Additionally, the nonlinear activation function produces various nonlinear transformations of the input, which are also stored and exploited as computational resources.

Other important features of RC include:
- Using an activation function for nonlinearity (similar to RNNs)
- Randomly fixing the recurrent connection weights
- Simply training the readout, resulting in a lower training cost compared to backpropagation
- Multitasking (multiplexing in some contexts) by reusing the same reservoir with multiple readouts

<div style="text-align: center; width: 750px; margin: auto; background-color: #f8f9fa; padding: 10px; border-radius: 10px;">

![fig_3_1.webp](../assets/fig_3_1.webp)

</div>

[figc]: #

[ja]: #
図1 (左)ESNの基本的構成。 (右) タコの腕を模したシリコンベースのソフトアームによる物理リザバー計算<sup>[3]</sup>。

[en]: #
Figure 1 (Left) Basic structure of ESN. (Right) An example of PRC platform with a silicone-based soft robotic arm inspired by the octopus<sup>[3]</sup>.

[END]: #

[/figc]: #

[ja]: #
内部結合の調整を伴わないランダムニューラルネットワークを使用するため、原理的には非線形性を有する大自由度のシステム (大自由度非線形力学系)であれば、リザバーの物理的実装の候補となります。
これらの実装は、実際に物理リザバー計算 (Physical Reservoir Computing; PRC)の分野で盛んに研究されています<sup>[3]</sup>。
例えば水中のタコの足の動きをセンシングし、その時系列データを計算資源として活用して、パリティービットチェックタスク等の非線形処理を実現できます。

[en]: #
Since it uses a random neural network without adjusting internal connections, any sufficiently nonlinear high-dimensional system can in principle serve as a physical reservoir.
These implementations are being actively studied in the field of physical reservoir computing (PRC)<sup>[3]</sup>.
For example, complicated dynamics obtained by sensing the movement of an octopus's arms underwater can be used to solve various nonlinear tasks by exploiting the intrinsic computational capability.

[ja]: #
### ESNの定式化

[en]: #
### Typical formulation of the ESN

[ja]: #

ここまで説明されたように、RCは入出力関係を学習する手法で、機械学習の一種である教師あり学習に分類されます。
学習の要点をまとめると下記のとおりになります。

0. 時系列入出力データの用意
1. 適切なESNモデルの選択ならびにハイパーパラメータの調整
2. 初期値の影響がなくなるまで系を時間発展 (ウォッシュアウト; washout)
3. 学習アルゴリズムの選択およびリードアウト層の調整
4. 入力データに対する出力データのサンプリング
5. 訓練誤差ならびに汎化誤差の評価

以下ESNの定式化のため、下記の表記を導入しながらより詳細を説明します。
なおわかりやすさのため以降、離散時間上で定義される時系列や関数には角括弧 $[\cdot]$、連続時間上で定義される時系列には丸括弧$(\cdot)$を用います。
教科書や論文等によっては丸括弧$(\cdot)$ や添字表記を用いる場合もありますが、意味は同じです。

- $k\in\mathbb{Z}$: 離散時間
- $u[k] \in \mathbb{R} $: 1次元入力
- $x[k] \in \mathbb{R}^{N}$: リザバーの内部状態
- $g: \mathbb{R}^{N}\to \mathbb{R}$: リードアウト層の関数
- $y[k] \in \mathbb{R} $: 入力時系列$\{u[0], u[1],~\ldots,~u[k]\}$ によって生成される目標出力

RCでは$\hat{y}[k]=g(x[k])\approx y[k]$ によって目標出力$y[k]$ の近似を目指します。
また、後の章でも改めて取り上げますが、$u[k]$ に対する**エコーステート性 (Echo State Property; ESP)** は、そのリザバーの情報処理能力を十分活用する上で重要な性質で、次の式を満たす系の性質を指します。

[en]: #
As explained so far, RC is a method of learning input-output relationships and is classified as supervised learning, a type of machine learning.
The main points are summarized as follows:

0. Prepare time-series input-output data.
1. Select the type of ESN model and design the hyperparameters.
2. Run the system until the effect of the initial state is washed out.
3. Select a learning algorithm and optimize the readout.
4. Sample the output according to the input data.
5. Evaluate the training and validation (generalization) errors.

The listed notation is used in the following description:
- $k\in\mathbb{Z}$: Discrete time
- $u[k]\in\mathbb{R}$: One-dimensional input
- $x[k]\in\mathbb{R}^N$: Reservoir states
- $g: \mathbb{R}^{N}\to \mathbb{R}$: Readout function
- $y[k]$: Target output generated by the input sequence $\{u[0], u[1],~\ldots,~u[k]\}$

We use square brackets $[\cdot]$ for time series or functions defined in discrete time and parentheses $(\cdot)$ for those defined in continuous time for clarity.
The aim of RC can be stated as approximating $y[k]$ by $\hat{y}[k]=g(x[k]).$
It should be noted that the reservoir should have the **echo state property (ESP)** for $u[k]$: there exists an **echo function** $\varphi$ that asymptotically determines the internal state from the input history:

[END]: #
$$
\begin{align*}
x[k] = \varphi(u[k], u[k-1],~\ldots)
.\end{align*}
$$

[ja]: #
換言すれば、ESPを満たす系は十分時間が経てばその内部状態の初期値に関わらず入力時系列$u[k]$によって内部状態がある関数$\varphi$で定まる値に収束します (漸近的に初期値$x[0]$の情報を消失)。
またこの$\varphi$は **エコー関数 (echo function)** と呼ばれます。

ここでESPはリザバーの要件として重要で、わかりやすさのためまず先に取り上げますが、必ずしも全てのRCにおいて必須ではない点に注意してください。
例えば後の章では初期値鋭敏性を有するカオス系を活用するRCについて学びますが、カオス系ではESPは満たされません。
また近年ではESPの拡張としてエコーインデックス (Echo Index; EI)が提案されており、全体としてESPは満たさないが、局所的に安定な応答を複数もつ系の場合にも対応しています<sup>[4]</sup>。
このようにESPを含めリザバーの要件に関する研究は現在も継続的に進められています。

[en]: #
In other words, the system asymptotically diminishes the effect of the initial conditions.

Although ESP is an important requirement for reservoirs and is introduced here for clarity, it is not necessarily essential for all RCs.
For example, in later chapters, we will study RCs that utilize chaotic systems with sensitivity to initial conditions, which do not satisfy ESP.
The echo index (EI) has been proposed as an extension of ESP, which can handle systems with multiple locally stable responses even if they do not satisfy ESP globally<sup>[4]</sup>.
In this way, research on reservoir requirements, including ESP, is ongoing.

[ja]: #
#### 入力層

[en]: #
#### Input layer

[ja]: #
- $W^\mathrm{in} \in \mathbb{R}^{N}$: 入力層の結合

$W^\mathrm{in}$は通常固定化され、しばしば一様分布$\mathcal{U}([-\sigma, \sigma])$を用いてランダムに生成されます。
またこの際使用される$\sigma\in \mathbb{R}$はしばしば入力スケールと呼ばれます。

[en]: #
- $W^\mathrm{in} \in \mathbb{R}^{N}$: Input weight (fixed)

Input weights are often randomly sampled from a uniform distribution $\mathcal{U}([-\sigma, \sigma])$, where $\sigma \in \mathbb{R}$ is known as the **input-scale** coefficient.

[ja]: #
#### リザバー層

[en]: #
#### Reservoir layer

[ja]: #
- $x[k] \in \mathbb{R}^{N}$: 時刻$t$における内部状態
- $W^\mathrm{rec} \in \mathbb{R}^{N\times N}$: 内部結合

$W^\mathrm{rec}$は多くの場合**スペクトル半径** $\rho(W^\mathrm{rec})$ によって制御されます。
これは$W^\mathrm{rec}$ の固有値の絶対値の最大値です (固有値が複素数を取りうる点に注意してください)。
このスペクトル半径の大小により入力に対する記憶特性が著しく変化します。

[en]: #
- $x[k] \in \mathbb{R}^{N}$: Internal units (reservoir states)
- $W^\mathrm{rec} \in \mathbb{R}^{N \times N}$: Internal weight matrix (fixed)

$W^\mathrm{rec}$ is often controlled by the **spectral radius** $\rho(W^\mathrm{rec})$, which is the maximum absolute value of the eigenvalue of $W^\mathrm{rec}$.

[ja]: #
#### リードアウト層 (1タスク)

[en]: #
#### Readout layer (for a single task)

[ja]: #
- $\hat{y}[k] \in \mathbb{R}$: 出力時系列
- $W^\mathrm{out} \in \mathbb{R}^{N + 1}$: 出力層の結合パラメータ

$W^\mathrm{out}$は学習データによって調整されます。
なお$N$ではなく$N+1$次元となっているのは、後述しますが定数項 (バイアス)に相当する項が増えているためです。

[en]: #
- $\hat{y}[k] \in \mathbb{R}$: Output time series
- $W^\mathrm{out} \in \mathbb{R}^{N + 1}$: Output weight (tuned)

Note that the dimension of the reservoir state is augmented with a bias of 1 for the optimization.

[ja]: #
#### ESNの時間発展と出力

[en]: #
#### Time evolution and output of ESN

[ja]: #
内部状態$x[k]$と出力$\hat{y}[k]$の時間発展は以下の式で表されます。

[en]: #
The internal state and output are defined as follows:

[END]: #
$$
\begin{align*}
x[k+1] &= f(W^\mathrm{rec}x[k] + W^\mathrm{in} u[k+1]) \\
\hat{y}[k] &= {W^\mathrm{out}}^\top [1 ; x[k]] \\
&= {W^\mathrm{out}}^\top {[1 \quad x_1[k] \quad \cdots \quad x_{N}[k]]}^\top \\
&= W^\mathrm{out}_0 + \sum_{i=1}^{N} W^\mathrm{out}_i x_{i}[k]
,\end{align*}
$$

[ja]: #
ここで$f:\mathbb{R}^N\to \mathbb{R}^N$は活性化関数と呼ばれ、ESNではよく$\tanh$が使用されます ($f(x)=[\tanh(x_1[k]) \quad \cdots \quad \tanh(x_N[k]) ]^\top$)。

[en]: #
with activation function e.g., $f(x)=[\tanh(x_1[k]) \quad \cdots \quad \tanh(x_N[k])]^\top$.

[ja]: #
#### その他の形式

[en]: #
#### Variations

[ja]: #
フィードバックループを明示的に導入できます。
これは閉ループ制御の文脈で登場します。

[en]: #
The setup can be extended with feedback, which will be used for closed-loop control:

[END]: #
$$
\begin{align*}
x[k+1] = f(W^\mathrm{rec}x[k] + W^\mathrm{in}u[k + 1] + W^\mathrm{fb}\hat{{y}}[k])
.\end{align*}
$$

[ja]: #
漏れ率 (leaky rate)を導入してより内部状態を保持しやすくしたリーキーESN (leaky ESN) も存在します。

[en]: #
Leaky (integrator) ESNs with a **leaky rate** are also used to control the dynamics' time scale:

[END]: #
$$
\begin{align*}
x[k+1] = (1-a)~x[k] + a~{f}(W^\mathrm{rec}x[k] + W^\mathrm{in}u[k+1])
.\end{align*}
$$

[ja]: #
## 演習問題と実演

[en]: #
## Exercises and demonstrations

[ja]: #
ここからは演習問題とデモンストレーションに移ります。
(離散) ESNを構築し、NARMA10と呼ばれる典型的なベンチマークタスクに取り組みましょう。
前章同様、まず次のセルを実行してください。

[en]: #
Now, let's build a discrete ESN and solve a typical benchmark task called NARMA10.
First, run the following cell as in the previous section.

In [ ]:
import math
import sys

import numpy as np

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/[[PROJECT_NAME]]/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/[[PROJECT_NAME]]/
    else:
        pass
        %cd /content/
        !git clone --branch [[BRANCH_NAME]] https://github.com/rc-bootcamp/[[PROJECT_NAME]].git
        %cd /content/[[PROJECT_NAME]]/
else:
    sys.path.append(".")

from utils.style_config import plt
from utils.tester import load_from_chapter_name
from utils.viewer import show_3d_coord

test_func, show_solution = load_from_chapter_name("03_esn_basics")

[ja]: #
### 1. 入力層・ESNの実装

[en]: #
### 1. Implement the basic structure of ESN

[ja]: #
まず初めに入力層に相当する`Linear`クラスと、リザバー層におけるESNを実装する`ESN`クラスを作成しましょう。

[en]: #
First, let's implement the `Linear` and the `ESN` classes, corresponding to the input and reservoir layers.

Q1.1.

[ja]: #
以下の穴埋めを実装し、結合行列$W\in \mathbb{R}^{N_\mathrm{out}\times N_\mathrm{in}}$とバイアス項$b\in\mathbb{R}^{N_\mathrm{out}}$を保持し、入力$u[k]\in\mathbb{R}^{... \times N_\mathrm{in}}$を線形変換して得られた多次元ベクトル$W u[k] + b \in \mathbb{R}^{... \times N_\mathrm{out}}$を出力するクラス`Linear`を完成させよ。
ただし$W$と$b$の要素はそれぞれ$\mathcal{U}([-\sigma_W, \sigma_W])$と$\mathcal{U}([-\sigma_b, \sigma_b])$ (引数中の`bound`と`bias`)で初期化される。

[en]: #
Fill in the blanks in the following code to complete the class `Linear`.
The class `Linear` holds a weight matrix $W\in \mathbb{R}^{N_\mathrm{out}\times N_\mathrm{in}}$ and a bias term $b\in\mathbb{R}^{N_\mathrm{out}}$.
Upon receiving an input $u[k]\in\mathbb{R}^{... \times N_\mathrm{in}}$, it outputs the linear transformation $W u[k] + b \in \mathbb{R}^{... \times N_\mathrm{out}}$.
The elements of $W$ and $b$ are initialized using uniform distributions $\mathcal{U}([-\sigma_W, \sigma_W])$ and $\mathcal{U}([-\sigma_b, \sigma_b])$ (denoted as `bound` and `bias` in the arguments), respectively.

[END]: #

- `Linear.__call__`
  - Argument(s):
    - `x`: `np.ndarray`
      - `shape`: `(..., input_dim)`
  - Return(s):
    - `out`: `np.ndarray`
      - `shape`: `(..., output_dim)`

[tips]: #
- [`np.swapaxes`](https://numpy.org/doc/stable/reference/generated/numpy.swapaxes.html)
- [`np.matmul`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html)

[/tips]: #

In [ ]:
class Module(object):
    def __init__(self, *_args, seed=None, rnd=None, dtype=np.float64, **_kwargs):
        if rnd is None:
            self.rnd = np.random.default_rng(seed)
        else:
            self.rnd = rnd
        self.dtype = dtype


class Linear(Module):
    def __init__(self, input_dim: int, output_dim: int, bound: float = None, bias: float = 0.0, **kwargs):
        """
        Linear model

        Args:
            input_dim (int): input dimension
            output_dim (int): output dimension
            bound (float, optional): sampling scale for weight. Defaults to None.
            bias (float, optional): sampling scale for bias. Defaults to 0.0.
        """
        super(Linear, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        if bound is None:
            bound = math.sqrt(1 / input_dim)
        self.weight = self.rnd.uniform(-bound, bound, (output_dim, input_dim)).astype(self.dtype)
        self.bias = self.rnd.uniform(-bias, bias, (output_dim,)).astype(self.dtype)

    def __call__(self, x: np.ndarray):
        x = np.asarray(x)
        out = np.matmul(x, self.weight.swapaxes(-1, -2)) + self.bias  # RIGHT Use `swapaxes` and `matmul`.
        return out


def solution(input_dim, output_dim, seed, bound, bias, mat):
    # DO NOT CHANGE HERE.
    lin = Linear(input_dim, output_dim, bound=bound, bias=bias, seed=seed)
    return lin(mat)


test_func(solution, "01_01")
# show_solution("01_01", "Linear")  # Uncomment it to see the solution.

Q1.2.

[ja]: #
次に以下の穴埋めを実装し、ESNの内部状態とその時間発展を制御する`ESN`クラスを完成させよ。
ただし、`ESN`はスペクトル半径が1になるように正規化された$W^\mathrm{rec}\in\mathbb{R}^{N\times N}~(\text{s.t.}~\rho(W^\mathrm{rec})=1)$と非線形パラメータ$\rho \in \mathbb{R}$を保持し、入力$v[k+1]$と内部状態$x[k]$に対する時間発展は以下の式で表される。

[en]: #
Next, fill in the blanks to complete the `ESN` class, which controls the internal state and its time evolution.
The `ESN` class holds an internal weight matrix $W^\mathrm{rec}\in\mathbb{R}^{N\times N}$ normalized so that its spectral radius equals 1 (i.e., $\rho(W^\mathrm{rec})=1$) and a nonlinear parameter $\rho \in \mathbb{R}$.
The time evolution of the reservoir state $x[k]$ given the input $v[k+1]$ is described by the following equation:

[END]: #
$$
\begin{align*}
x[k+1] = f(\rho W^\mathrm{rec} x[k] + v[k+1])
.\end{align*}
$$

- `ESN.__call__`
  - Argument(s):
    - `x`: `np.ndarray`
      - `shape`: `(..., dim)`
    - `v`: `np.ndarray`
      - `shape`: `(..., dim)`
  - Return(s):
    - `x_next`: `np.ndarray`:
      - `shape`: `(..., dim)`

[tips]: #

[ja]: #
- [`scipy.sparse.linalg.eigs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html)を使用せよ。
  - 再現性のため、`v0=np.ones((dim,))`を指定せよ。
  - 高速化のため、`which='LM', k=1`を指定せよ。
- [Spectral radius](https://en.wikipedia.org/wiki/Spectral_radius)

[en]: #
- Use [`scipy.sparse.linalg.eigs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html).
  - For reproducibility, specify `v0=np.ones((dim,))`.
  - For speed, specify `which='LM', k=1`.
- [Spectral radius](https://en.wikipedia.org/wiki/Spectral_radius)

[END]: #

[/tips]: #

In [ ]:
from scipy.sparse.linalg import ArpackNoConvergence, eigs


class ESN(Module):
    def __init__(
        self,
        dim: int,
        sr: float = 1.0,
        f=np.tanh,
        a: float | None = None,
        p: float = 1.0,
        init_state: np.ndarray | None = None,
        normalize: bool = True,
        **kwargs,
    ):
        """
        Echo state network [Jaeger, H. (2001). Bonn, Germany:
        German National Research Center for Information Technology GMD Technical Report, 148(34), 13.]

        Args:
            dim (int): number of the ESN nodes
            sr (float, optional): spectral radius. Defaults to 1.0.
            f (callable, optional): activation function. Defaults to np.tanh.
            a (float | None, optional): leaky rate. Defaults to None.
            p (float, optional): density of connection matrix. Defaults to 1.0.
            init_state (np.ndarray | None, optional): initial states. Defaults to None.
            normalize (bool, optional): decide if normalizing connection matrix. Defaults to True.
        """
        super(ESN, self).__init__(**kwargs)
        self.dim = dim
        self.sr = sr
        self.f = f
        self.a = a
        self.p = p
        if init_state is None:
            self.x_init = np.zeros(dim, dtype=self.dtype)
        else:
            self.x_init = np.array(init_state, dtype=self.dtype)
        self.x = np.array(self.x_init)
        # Generating normalized sparse matrix
        while True:
            try:
                self.weight = self.rnd.normal(size=(self.dim, self.dim)).astype(self.dtype)
                if self.p < 1.0:  # Sparse matrix
                    # BEGIN (Optional!) Implement sparse matrix with density `self.p`.
                    w_con = np.full((dim * dim,), False)
                    w_con[: int(dim * dim * self.p)] = True
                    w_con = w_con.reshape((dim, dim))
                    self.rnd.shuffle(w_con)
                    self.weight = self.weight * w_con
                    # END
                if normalize:
                    # RIGHT_B Calculate `spectral_radius`. Use `eigs` and specify `v0` parameter as `np.ones(self.dim)` to ensure the reproducibility.
                    eigen_values = eigs(self.weight, return_eigenvectors=False, k=1, which="LM", v0=np.ones(self.dim))
                    spectral_radius = max(abs(eigen_values))
                    # RIGHT_E
                    self.weight = self.weight / spectral_radius  # RIGHT Normalize the weight matrix.
                break
            except ArpackNoConvergence:
                continue

    def __call__(self, x: np.ndarray, v: np.ndarray | None = None):
        x_next = self.sr * np.matmul(x, self.weight.swapaxes(-1, -2))  # RIGHT Calculate the next state `x_next`.
        if v is not None:
            x_next += v  # RIGHT Add the input `v` if it is given.
        x_next = self.f(x_next)  # RIGHT Apply the activation function.
        if self.a is None:
            return x_next
        else:
            return (1 - self.a) * x + self.a * x_next

    def step(self, v: np.ndarray | None = None):
        self.x = self(self.x, v)


def solution(dim, sr, seed, xmat, vmat):
    # DO NOT CHANGE HERE.
    net = ESN(dim, sr=sr, f=np.tanh, seed=seed)
    return net(xmat, v=vmat)


test_func(solution, "01_02")
# show_solution("01_02", "ESN")  # Uncomment it to see the solution.

[ja]: #
次のコードにより`ESN.w_net`のスペクトル半径が1以下すなわち、固有値が単位円内に収まっているかどうか確認しましょう。

[en]: #
Check if the ESN is successfully normalized by the following code, which visualizes the distribution of the eigenvalues of `ESN.w_net`.

[tips]: #

[ja]: #
- [円則](https://ja.wikipedia.org/wiki/%E3%83%A9%E3%83%B3%E3%83%80%E3%83%A0%E8%A1%8C%E5%88%97#%E5%86%86%E5%89%87)

[en]: #
- [Circular law](https://en.wikipedia.org/wiki/Circular_law)

[/tips]: #

In [ ]:
def show_eigen(mat):
    assert mat.ndim == 2
    assert mat.shape[0] == mat.shape[1]
    ts = np.linspace(0, 2 * np.pi, 100001)
    es = np.linalg.eigvals(mat)
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.scatter(np.real(es), np.imag(es), s=3.0)
    ax.plot(np.cos(ts), np.sin(ts), lw=1.0, ls=":", color="k")
    ax.set_xlabel("real")
    ax.set_ylabel("imag")


net = ESN(500, sr=1.0, seed=1234)
show_eigen(net.weight)

Q1.3. (Advanced)

[ja]: #
- 上記のコードで密度$p$ (`math.floor(self.dim * self.dim * p)`個の要素だけ非零) のスパース行列を作成するコードが穴埋めになっている。同様に自力での実装を試みよ。
- `normalize=False`を指定し、固有値の分布がどうなるか確認せよ。またスパース ($p < 1$)なケースでの変化も確認せよ。
- ノード数が非常に大きい時 (e.g., $N > 10^4$)、固有値の計算は非常に時間がかかるため、`normalize=True`の指定は現実的でない。この際、ノード数$N$と結合密度$p$から、おおよそのスペクトル半径が$1$になる定数倍の係数を求めよ。
ただし検証は`normalize=False`を用いて行え。

[en]: #
- The above code contains a fill-in-the-blanks section for creating a sparse matrix with density $p$ (i.e. only `math.floor(self.dim * self.dim * p)` elements are non-zero).
Try implementing this by yourself.
- Check how the distribution of eigenvalues changes when you set `normalize=False`.
Likewise, check the changes in the case of using a sparse matrix ($p < 1$).
- When the number of nodes is very large (e.g., $N > 10^4$), calculating eigenvalues becomes very time-consuming, making the use of `normalize=True` impractical.
In this case, derive a constant multiplier coefficient that scales the spectral radius to approximately $1$, based on the number of nodes $N$ and the density $p$.
Verify this by setting `normalize=False`.

[ja]: #
### 2. ウォッシュアウト

[en]: #
### 2. Washout phase

[ja]: #
**ウォッシュアウト** (内部状態の「洗い流し」)は、初期条件の影響を消失させるために導入されます。
ウォッシュアウトにより、システムが同じ入力に対して同じ内部状態を再現的に取るようになり結果的に計算能力の向上が見込めます。

- $T_\mathrm{washout}$: ウォッシュアウトの時間ステップ
- $T_\mathrm{sample}$: その後サンプリングする時間ステップ
- $U=\{u[-T_\mathrm{washout}],~\ldots,~u[0],~\ldots,~u[T_\mathrm{sample} - 1] \}$: 入力時系列

先程実装した`ESN`と`Linear`を用いてウォッシュアウトの効果を確かめてみましょう。
以下のコードは一様乱数$\mathcal{U}([-1, 1])$からサンプルされた同じ入力$u[k]$に対する`sample_num`個の異なる初期値のESNの応答を表示するコードです。

[en]: #
The **washout phase** is introduced to diminish the effect of the initial condition, enabling the system to reproduce the same internal states and thereby enhancing computational capability.

- $T_\mathrm{washout}$: Washout length
- $T_\mathrm{sample}$: Sampling length
- $U=\{u[-T_\mathrm{washout}],~\ldots,~u[0],~\ldots,~u[T_\mathrm{sample} - 1] \}$: Input sequence

Let's check the effect of washout using the `ESN` and `Linear` classes implemented earlier.
The following code displays the ESN responses for `sample_num` different initial values given the same input $u[k]$ sampled from the uniform distribution $\mathcal{U}([-1, 1])$.

In [ ]:
rnd = np.random.default_rng(1234)

dim_in = 1
dim_esn = 100  # The number of ESN nodes
sample_num = 3  # The number of randomly-sampled initial conditions
t_washout, t_sample = 100, 1000
t_total = t_washout + t_sample

ts = np.arange(-t_washout, t_sample)
us = rnd.uniform(-1, 1, (t_total, dim_in))  # Input sequence
x_init = rnd.uniform(-1, 1, (sample_num, dim_esn))  # Initial conditions

w_in = Linear(dim_in, dim_esn, bound=0.1, bias=0.0, rnd=rnd)
net = ESN(dim_esn, sr=0.995, f=np.tanh, rnd=rnd, init_state=x_init)

x = x_init
xs = np.zeros((t_total, *x_init.shape))  # Shape: (t_total, sample_num, dim_esn)
for idx, _t in enumerate(ts):
    x = net(x, w_in(us[idx]))
    xs[idx] = x

[ja]: #
`xs`を下のコードで表示してみましょう。

[en]: #
Let's visualize `xs` with the following code.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

ax[0, 0].plot(ts[:t_washout], xs[:t_washout, :, 0])
ax[1, 0].plot(ts[:t_washout], xs[:t_washout, :].mean(axis=-1))
ax[0, 1].plot(ts[-100:], xs[-100:, :, 0])
ax[1, 1].plot(ts[-100:], xs[-100:, :].mean(axis=-1))
ax[0, 0].set_ylabel(r"$x_0[k]$")
ax[1, 0].set_ylabel(r"$\bar{x}[k]$")
ax[0, 0].set_title(r"$k\in[-T_\mathrm{washout}, 0)$")
ax[0, 1].set_title(r"$k\in[T_\mathrm{sample} - 100, T_\mathrm{sample})$")

None

[ja]: #
以下のコードは、最初の3ノード$x_0, x_1, x_2$で3次元座標を構築し、その軌跡を表示します。
まずは$k\in[-T_\mathrm{washout}, 0)$の区間のみ表示します。

[en]: #
The following code constructs a 3D coordinate using the first 3 nodes $x_0, x_1, x_2$ and displays their trajectory for each initial condition.
First, let's display the dynamics for $k\in[-T_\mathrm{washout}, 0)$.

In [ ]:
fig = show_3d_coord(
    xs[:t_washout, 0, :3],
    xs[:t_washout, 1, :3],
    xs[:t_washout, 2, :3],
    axes=["x_{}[k]".format(idx) for idx in range(3)],
)
fig.show()

[ja]: #
$k\in[T_\mathrm{sample} - 100, T_\mathrm{sample})$の区間は以下のとおりです。
ほとんど重なっている様子が見えます。

[en]: #
Next, let's display ones during $k\in[T_\mathrm{sample} - 100, T_\mathrm{sample})$.
You will see that they almost overlap.

In [ ]:
fig = show_3d_coord(
    xs[-100:, 0, :3],
    xs[-100:, 1, :3],
    xs[-100:, 2, :3],
    axes=["x_{}[k]".format(idx) for idx in range(3)],
)
fig.show()

Q2.1. (Advanced)

[ja]: #
他のパラメータを変えて同様にウォッシュアウトの効果を確認しESPに対する理解を深めよう。
- `sr`を変えよ。特に1以上のケースを確認せよ。
- `bound`を変えてみよ。
- `esn_dim`を大きくさせてみよ。
- 異なる活性化関数`f`を試せ。

[en]: #
Check the effect of the washout phase for different parameter values and deepen your understanding of the ESP.
- Change the value of `sr`, especially for cases where it is larger than 1.
- Change the value of `bound`.
- Try increasing `esn_dim`.
- Try different activation functions `f`.

[ja]: #
### 3. 学習: 線形回帰によるリードアウト層の学習

[en]: #
### 3. Training phase: Training readout weight by linear regression

[ja]: #
学習は線形回帰によりワンショット方式で達成されます。
このような学習方式は**オフライン学習** (offline training)と呼ばれます。

- $T_\mathrm{train}$: 学習の時間ステップ
- $\{u[0],~\ldots,~u[T_\mathrm{train} - 1]\}$: 入力時系列
- $\{y[0],~\ldots,~y[T_\mathrm{train}- 1]\}$: 目標時系列

[en]: #
Training can be done in one shot using linear regression.
This scheme of training is called **offline training**.

- $T_\mathrm{train}$: Training length
- $\{u[0],~\ldots,~u[T_\mathrm{train} - 1]\}$: Input sequence
- $\{y[0],~\ldots,~y[T_\mathrm{train} - 1]\}$: Target sequence

[ja]: #
タスク$y[k]$のコスト関数$\mathrm{RSS}$ は次のように定義されます。

[en]: #
The cost function $\mathrm{RSS}$ for a single task $y[k]$ can be defined as follows:

[END]: #

$$
\mathrm{RSS} := \sum_{k=0}^{T_\mathrm{train}-1} \|  y[k] - \hat{y}[k] \|^2,
$$

[ja]: #
さらに以下のように展開されます。

[en]: #
which can be expanded as follows:

[END]: #
$$
\begin{align*}
\mathrm{RSS} &= \sum_{k=0}^{T_\mathrm{train}-1} \| y[k] - \hat{y}[k] \|^2 \\
&=  \sum_{k=0}^{T_\mathrm{train}-1} \| y[k] - [1: x[k]]w^\mathrm{out} \|^2 \\
&= \| y^\mathrm{train} - X^\mathrm{train}w^\mathrm{out} \|^2
,\end{align*}
$$

[ja]: #
ただし

[en]: #
where

[END]: #
$$
\begin{align*}
X^\mathrm{train} &:= \begin{bmatrix}
1 & x_{0}[0] &\cdots & x_{N-1}[0] \\
\vdots & \vdots & \ddots & \vdots \\
1 & x_{0}[T_\mathrm{train}-1] & \cdots & x_{N-1}[T_\mathrm{train}-1]
\end{bmatrix} \in \mathbb{R}^{T_\mathrm{train} \times (N + 1)} \\
y^\mathrm{train} &:= [y[0] \quad \cdots \quad y[T_\mathrm{train}-1]]^\top \in \mathbb{R}^{T_\mathrm{train}} \\
w^\mathrm{out} &:= [{w}^\mathrm{out}_{0} \quad {w}^\mathrm{out}_{1} \quad \cdots \quad {w}^\mathrm{out}_{N} ]^\top \in \mathbb{R}^{N+1}
.\end{align*}
$$

[ja]: #
${X^\mathrm{train}}^\top X^\mathrm{train}$ の階数が最大 (full-rank) の場合、$\mathrm{RSS}$ を最小化する最適解は次の式で得られます。

[en]: #
When ${X^\mathrm{train}}^\top X^\mathrm{train}$ is full-rank, optimal solution minimizing $\mathrm{RSS}$ can be obtained by the following equation:

[END]: #
$$
\hat{w}^\mathrm{out} = \arg \min_{w^\mathrm{out}}\mathrm{RSS} = ({X^\mathrm{train}}^\top X^\mathrm{train})^{-1}{X^\mathrm{train}}^\top y^\mathrm{train},
$$

[ja]: #
最後に、評価フェーズの出力 ($t \in [T_\mathrm{train}, T_\mathrm{train}+T_\mathrm{eval}-1] $) は次のように記述されます。

[en]: #
Finally, the output in evaluation phase ($t \in [T_\mathrm{train}, T_\mathrm{train}+T_\mathrm{eval}-1] $) is written as

[END]: #
$$
\begin{align*}
\hat{y}[k] &= [1:x[k]]\hat{w}^\mathrm{out} \\
\hat{y} &= X^\mathrm{eval}\hat{w}^\mathrm{out}
,\end{align*}
$$

[ja]: #
ただし

[en]: #
where

[END]: #
$$
X^\mathrm{eval} =
\begin{bmatrix} 1 & x_{0}[T_\mathrm{train}] &\cdots & x_{N-1}[T_\mathrm{train}] \\
 \vdots & \vdots & \ddots & \vdots \\
  1 & x_{0}[T_\mathrm{train}+T_\mathrm{eval}-1] & \cdots & x_{N-1}[T_\mathrm{train}+T_\mathrm{eval}-1]
\end{bmatrix} \in \mathbb{R}^{T_\mathrm{eval} \times (N + 1)}.
$$

Q3.1.

[ja]: #
先程実装した`Linear`を継承し、与えられた$X \in \mathbb{R}^{T\times N^\mathrm{in}}, Y \in \mathbb{R}^{T \times N^\mathrm{out}}$に対して上記の線形回帰を実行し、得られた$\hat{w}^\mathrm{out}$を用いて、重みとバイアスを更新する`LRReadout`を実装せよ。

[en]: #
Implement the `LRReadout` class that inherits the previously implemented class `Linear`.
This class should perform linear regression on the given $X \in \mathbb{R}^{T \times N^\mathrm{in}}$ and $Y \in \mathbb{R}^{T \times N^\mathrm{out}}$, and update the weights and biases with the obtained $\hat{w}^\mathrm{out}$.

[END]: #
- `LRReadout.train`
  - Argument(s):
    - `x`: `np.ndarray`
      - `shape`: `(time_steps, input_dim)`
    - `y`: `np.ndarray`
      - `shape`: `(time_steps, output_dim)`
  - Return(s):
    - `self.weight`: `np.ndarray`
      - `shape`: `(output_dim, input_dim)`
    - `self.bias`: `np.ndarray`
      - `shape`: `(output_dim,)`

[ja]: #
  - Operation(s):
      - `self.weight`の更新
      - `self.bias`の更新

[en]: #
  - Operation(s):
      - Update `self.weight` with the obtained weight.
      - Update `self.bias` with the obtained bias.

[END]: #

[tips]: #
- [`np.linalg.lstsq`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html)

[/tips]: #

In [ ]:
class LRReadout(Linear):
    def train(self, x: np.ndarray, y: np.ndarray):
        assert (x.ndim == 2) and (x.shape[-1] == self.input_dim)
        assert (y.ndim == 2) and (y.shape[-1] == self.output_dim)
        # BEGIN Use `lstsq` to calculate the optimal weight and bias.
        x_biased = np.ones((x.shape[0], x.shape[1] + 1), dtype=self.dtype)
        x_biased[:, 1:] = x
        sol, _residuals, _rank, _s = np.linalg.lstsq(x_biased, y, rcond=None)
        self.weight[:] = sol[1:].T
        self.bias[:] = sol[0]
        # END
        return self.weight, self.bias


def solution(dim_in, dim_out, x_train, y_train, x_eval):
    # DO NOT CHANGE HERE.
    readout = LRReadout(dim_in, dim_out)
    readout.train(x_train, y_train)
    return readout(x_eval)


test_func(solution, "03_01")
# show_solution("03_01", "LRReadout")  # Uncomment it to see the solution.

[ja]: #
### 4. NARMA10―10次の非線形自己回帰移動平均<sup>[5]</sup>

[en]: #
### 4. NARMA10―The 10th-order nonlinear autogressive moving average<sup>[5]</sup>

[ja]: #
NARMA (Nonlinear AutoRegressive Moving Average) 10<sup>[5]</sup> は、RC で頻繁に使用される典型的なベンチマークです。
自己回帰部分と移動平均部分で構成される非線形 ARMA (AutoRressive Moving Average)モデルであり、現在と過去値の間には非線形な関係があります。
典型的には一様ランダム入力 $u[k]\sim\mathcal{U}([-1,1])$に対して、NARMA10 モデルは次のように定義されます。

[en]: #
NARMA (Nonlinear AutoRegressive Moving Average) 10<sup>[5]</sup> is a typical benchmark task frequently used in RC.
It consists of an autoregressive part and a moving average part, meaning that the value depends on past inputs in a nonlinear manner.
Given a uniform random input $u[k] \sim \mathcal{U}([-1, 1])$, the NARMA10 model is defined as follows:

[END]: #
$$
\begin{align*}
y[k+1] &= \alpha y[k] + \beta y[k]\sum_{i=0}^{9}y[k-i] + \gamma \nu[k-9]\nu[k] + \delta \\
\nu[k]&=\mu u[k]+\kappa
,\end{align*}
$$

[ja]: #
ここで、$(\alpha,\beta,\gamma,\delta,\mu,\kappa)=(0.3,0.05,1.5,0.1,0.25,0.25)$がよく用いられます。
値の発散を避けるために、入力 $u[k]$ は慎重に調整されます。
特に$(\mu,\kappa)=(0.25,0.25)$により$\nu[k] \sim \mathcal{U}([0.0, 0.5])$ にスケーリングされる場合が多いです。

[en]: #
where $(\alpha,\beta,\gamma,\delta,\mu,\kappa)=(0.3,0.05,1.5,0.1,0.25,0.25)$, and
the input $u[k]$ is biased and scaled so that $\nu[k] \sim \mathcal{U}([0.0,0.5])$ to avoid divergence.

Q4.1.

[ja]: #
長さ$T + 10$の入力時系列 $U=\{u[-10],~\ldots,~u[T-1]\}\in\mathbb{R}^{(T+10)\times d}$、
長さ10の時系列 $Y^\mathrm{init}=\{y[-10],~\ldots,~y[-1]\}\in \mathbb{R}^{10 \times d}$
が与えられる。
NARMA10
上記の式に従い変換を施したNARMA10の時系列 $Y=\{y[-10],~\ldots,~y[0],~\ldots,~y[T-1]\}\in\mathbb{R}^{T\times d}$ を出力する関数`narma_func`を実装せよ。

[en]: #
An input time series $U = \{u[-10],~\ldots,~u[T-1]\}\in\mathbb{R}^{(T+10)\times d}$ with length $T + 10$ and another time series $Y^\mathrm{init}=\{y[-10],~\ldots,~y[-1]\}\in \mathbb{R}^{10 \times d}$ with length 10 are given.
Implement the function `narma_func` that outputs the NARMA10 time series $Y=\{y[-10],~\ldots,~y[0],~\ldots,~y[T-1]\}\in\mathbb{R}^{T\times d}$, following the equations defined above.

[END]: #

- `narma_func`
  - Argument(s):
    - `us`: `np.ndarray`
      - `shape`: `(t + 10, d)`
    - `y_init`: `np.ndarray`
      - `shape`: `(10, d)`
  - Return(s):
    - `ys`: `np.ndarray`
      - `shape`: `(t + 10, d)`

[tips]: #
- [`np.sum`](https://numpy.org/doc/stable/reference/generated/numpy.sum.html)

[/tips]: #

In [ ]:
def narma_func(us, y_init, alpha=0.3, beta=0.05, gamma=1.5, delta=0.1, mu=0.25, kappa=0.25):
    assert us.shape[0] > 10
    assert y_init.shape[0] == 10
    assert y_init.shape[1:] == us.shape[1:]
    vs = mu * us + kappa
    ys = np.zeros_like(vs)
    # BEGIN Implement NARMA10 system.
    ys[:10] = y_init
    for idx in range(10, ys.shape[0]):
        ys[idx] += alpha * ys[idx - 1]
        ys[idx] += beta * ys[idx - 1] * np.sum(ys[idx - 10 : idx], axis=0)
        ys[idx] += gamma * vs[idx - 10] * vs[idx - 1]
        ys[idx] += delta
    # END
    return ys


test_func(narma_func, "04_01")
# show_solution("04_01", "narma_func")  # Uncomment it to see the solution.

[ja]: #
以下のコードで実際にNARMA10の時系列を表示させてみましょう。

[en]: #
The following code displays the dynamics of NARMA10 for input $u[k]$:

In [ ]:
rnd = np.random.default_rng(1234)

length = 200

us = rnd.uniform(-1.0, 1.0, length + 10)
y_init = np.zeros(10)
ys = narma_func(us, y_init)

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.plot(us * 0.25 + 0.25, label=r"$v[k]$", ls=":", color="k")
ax.plot(ys, label="NARMA10")
ax.legend(loc="upper left", bbox_to_anchor=(1.025, 1.0), borderaxespad=0, ncol=1)

Q4.2. (Advanced)

[ja]: #
NARMA10は発散しやすいベンチマークタスクであり、そのパラメータの取り扱いには慎重を要する。
- スケーリングを二倍、すなわち$u[k] \in [-2, 2]$で値の発散を確認せよ。
- サンプリング期間を非常に大きくする($T\approx 10^6$)とかなりの確率で値の発散が発生する。これを確認せよ。
- 発散しないための工夫としてクリッピングが考えられる。典型的には$\tanh$によって$y[k]$の値を毎ステップ制限する方策が取られる。この処置を実装し実際に発散の軽減を確認せよ。

[en]: #
The NARMA10 benchmark task is prone to divergence, so its parameters must be handled carefully.
- Check the divergence by doubling the scale of input (i.e., $u[k] \in [-2, 2]$).
- Divergence is more likely to occur when the sampling period is very large ($T\approx 10^6$).
Confirm this behavior.
- Clipping is an effective measure to prevent divergence.
For example, you may use $\tanh$ to limit the values of $y[k]$ at every time step.
Implement this measure and confirm that it reduces the occurrence of divergence.

[ja]: #
### 5. 精度の評価

[en]: #
### 5. Evaluation of the performance

[ja]: #
- $T_\mathrm{in}$: 評価時間ステップ

平均二乗平方根誤差 (RMSE) と正規二乗平均平方根誤差 (NRMSE) は次の式から定義されます。

[en]: #
- $T_\mathrm{in}$: Evaluation length

The root mean squared error (RMSE) and normalized root mean squared error (NRMSE) can be defined as follows:

[END]: #
$$
\begin{align*}
\mathrm{NRMSE}(y^\mathrm{eval}, \hat{y}^\mathrm{eval}) :&= \dfrac{\mathrm{RMSE}(y^\mathrm{eval}, \hat{y}^\mathrm{eval}) }{\sigma(y^\mathrm{eval})} \\
\mathrm{RMSE}(y^\mathrm{eval}, \hat{y}^\mathrm{eval}) :&=  \sqrt{\dfrac{\mathrm{RSS}(y^\mathrm{eval}, \hat{y}^\mathrm{eval}) }{T_\mathrm{eval}} } \\
&= \sqrt{ \dfrac{1}{T_\mathrm{eval}} \sum_{k=T_\mathrm{train}}^{T_\mathrm{train}+T_\mathrm{eval}-1} ( y[k] - \hat{y}[k])^2}
,\end{align*}
$$

[ja]: #
ここで、$\sigma^2(y^\mathrm{eval})$ は出力 $y^\mathrm{eval}=\{y[T_\mathrm{train}],~\ldots,~y[T_\mathrm{train}+T_\mathrm{eval}-1]\}$ の分散を表します。
状況によりますが、$\mathrm{NRMSE} < 0.2$ が精度が良好か判別する基準として一つの目安となります。

[en]: #
where $\sigma^2(y^\mathrm{eval})$ is the variance of $y^\mathrm{eval}=\{y[T_\mathrm{train}],~\ldots,~y[T_\mathrm{train}+T_\mathrm{eval}-1]\}$ through time.
As a rough guideline, $\mathrm{NRMSE} < 0.2$ often indicates good performance.

Q5.1.

[ja]: #
時間$T$ステップにわたり$d$個の時系列を記録した$Y\in\mathbb{R}^{T\times d}$と$\hat{Y}\in\mathbb{R}^{T\times d}$が与えられる。
上記の式を参考に各$d$要素について$\mathrm{NRMSE}$を計算した、$\mathrm{NRMSE}(Y, \hat{Y})\in \mathbb{R}^{d}$を出力するコードを書け。

[en]: #
Given two matrices $Y\in\mathbb{R}^{T\times d}$ and $\hat{Y}\in\mathbb{R}^{T\times d}$, each containing $d$ time series measured over $T$ time steps, write code to compute the $\mathrm{NRMSE}$ for each of the $d$ elements.
The output $\mathrm{NRMSE}(Y, \hat{Y})\in \mathbb{R}^{d}$ should be calculated based on the above equations.

[END]: #

- `calc_nrmse`
  - Argument(s):
    - `y`: `np.ndarray`
      - `shape`: `(t, d)`
    - `yhat`: `np.ndarray`
      - `shape`: `(t, d)`
  - Return(s):
    - `nrmse`: `np.ndarray`
      - `shape`: `(d,)`

[tips]: #
- [`np.mean`](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
- [`np.var`](https://numpy.org/doc/stable/reference/generated/numpy.var.html)

[/tips]: #

In [ ]:
def calc_nrmse(y, yhat):
    # BEGIN
    mse = y - yhat
    mse = (mse * mse).mean(axis=0)
    var = y.var(axis=0)
    return (mse / var) ** 0.5
    # END


test_func(calc_nrmse, "05_01")
# show_solution("05_01", "calc_nrmse")  # Uncomment it to see the solution.

[ja]: #
以下のサンプルコードによって、下記の典型的な設定でNARMA10を100ノードのESNに解かせてみましょう。
これまで実装した`Linear`・`ESN`・`LRReadout`・`narma_func`・`calc_nrmse`を使用します。

[en]: #
Let's use the following sample code to let 100-node ESN solve NARMA10 with the typical settings listed below.
The implemented modules `Linear`, `ESN`, `LRReadout`, `narma_func`, and `calc_nrmse` will be used below.

[END]: #

- $(N, \rho, \sigma) = (100, 0.9, 0.1)$
- $(T_\mathrm{washout}, T_\mathrm{train}, T_\mathrm{eval}) = (100, 2000, 1000)$
- $(\alpha,\beta,\gamma,\delta,\mu,\kappa)=(0.3,0.05,1.5,0.1,0.25,0.25)$

In [ ]:
rnd = np.random.default_rng(1234)
dim_esn, rho, sigma = 100, 0.9, 0.1
t_washout, t_train, t_eval = 100, 2000, 1000
narma_parameters = dict(alpha=0.3, beta=0.05, gamma=1.5, delta=0.1, mu=0.25, kappa=0.25)

t_total = t_washout + t_train + t_eval
ts = np.arange(-t_washout, t_train + t_eval)
us = rnd.uniform(-1, 1, (t_total, 1))
ys = narma_func(us, np.zeros((10, 1)), **narma_parameters)
x_init = rnd.uniform(-1, 1, (dim_esn,))
w_in = Linear(1, dim_esn, bound=sigma, bias=0.0, rnd=rnd)
net = ESN(dim_esn, sr=rho, f=np.tanh, init_state=x_init, rnd=rnd)
w_out = LRReadout(dim_esn, 1)

x = x_init
xs = np.zeros((t_total, dim_esn))
for idx in range(t_total):
    x = net(x, w_in(us[idx]))
    xs[idx] = x

# Or, you can write as follows in the OOP style:
# xs = np.zeros((t_total, dim_esn))
# for idx in range(t_total):
#     net.step(w_in(us[idx]))
#     xs[idx] = net.x

x_train, y_train = xs[t_washout:-t_eval], ys[t_washout:-t_eval]
x_eval, y_eval = xs[-t_eval:], ys[-t_eval:]
w_out.train(x_train, y_train)
y_out = w_out(x_eval)
nrmse = calc_nrmse(y_eval, y_out)

plot_length = 200

fig, ax = plt.subplots(3, 1, figsize=(12, 8), gridspec_kw={"hspace": 0.05})
ax[0].plot(ts[-plot_length:], us[-plot_length:])
ax[0].set_xticklabels([])
ax[1].plot(ts[-plot_length:], xs[-plot_length:, :10], lw=1.0)
ax[1].set_xticklabels([])
ax[2].plot(ts[-plot_length:], y_eval[-plot_length:], color="k", ls=":", lw=1.5)
ax[2].plot(ts[-plot_length:], y_out[-plot_length:], lw=1.5, color="red")
ax[0].set_title("NRMSE={:.3e}".format(nrmse[0]))
ax[0].set_ylabel(r"$u[k]$")
ax[1].set_ylabel(r"$x[k]$")
ax[2].set_ylabel(r"$y[k]$ & $\hat{y}[k]$")
ax[2].set_xlabel("time steps")

None

Q5.2. (Advanced)

[ja]: #
NARMA10の精度は上記のパラメータの選択によって大きく変化する。
「良い」(目安は$\mathrm{NRMSE}<0.2$)となるようなパラメータセットを探すべく以下のパターンを試せ。
- 係数$0.5$と定数項$0.5$ (`w_in(0.5 * u + 0.5)`)を採用し、実質的に非対称入力$u[k]\sim \mathcal{U}([0, 1])$を実現せよ。またその時の精度の変化を計測せよ。
- ノード数を変化させよ。特に $N(=100, 200,~\ldots,~1000)$ のケースを試し、横軸 $N$・縦軸 $\mathrm{NRMSE}$のグラフとして描画せよ。
- 学習用のサンプリング時間を長くせよ。特に $T_\mathrm{train}=(2000, 4000,~\ldots,~20000)$ と変化させ同様に、横軸 $T_\mathrm{train}$ ・縦軸 $\mathrm{NRMSE}$ のグラフを描画せよ。
- $(\mu, \kappa)=(0.5, 0.0)$ により $\nu[k] \sim \mathcal{U}([-0.5, 0.5])$ として同様に精度を計測せよ。

[en]: #
The accuracy of NARMA10 varies significantly depending on the parameter selection.
Follow the instructions below to find a "good" parameter set ($\mathrm{NRMSE}<0.2$ is a reasonable benchmark).

- Adopt a coefficient of $0.5$ and a bias of $0.5$ (`w_in(0.5 * u + 0.5)`), which effectively uses an asymmetric input $u[k] \sim \mathcal{U}([0, 1])$.
Observe how the accuracy changes in this setting.
- Change the number of nodes.
Specifically, test cases of $N = 100, 200,~\ldots,~1000$, and plot a graph with $N$ on the x-axis and $\mathrm{NRMSE}$ on the y-axis.
- Change the sampling length.
Specifically, test cases of $T_\mathrm{train} = 2000, 4000,~\ldots,~20000$, and plot a graph with $T_\mathrm{train}$ on the x-axis and $\mathrm{NRMSE}$ on the y-axis.
- Adopt $(\mu, \kappa) = (0.5, 0.0)$ to use $\nu[k] \sim \mathcal{U}([-0.5, 0.5])$, and observe how the accuracy changes in this setting.

Q5.3. (Advanced)

[ja]: #
次章の内容で取り扱うが、パラメータの体系的な探索は**グリッドサーチ (grid search)**と呼ばれ、RCに限らず解析や最適化の場面において幅広く登場する。
典型的には`for`文を用いた多重ループが考えられるが、前回学んだとおりPythonではfor文の実行は遅く、探索するパラメータ数が増大するとそのオーバーヘッドが無視できなくなる。
一方でNumPyは、扱うテンソルのサイズが大きいほど一般に並列化の恩恵を受け効率的であるといえる。
`for`文を極力用いずに (理想的には`ESN`の時間発展のみに使用)効率的にグリッドサーチを行う方法を考えよ。
特に以下のケースに関して考察せよ。
- $\sigma$: 入力スケール
- $\rho$: スペクトル半径
- $\mu, \kappa$: NARMA10のパラメータ

なおここに挙げられたパラメータに関するグリッドサーチは、今回実装した関数に変更を**加えずに**数行のコードの追加で実現できる。

[en]: #
In the next chapter, we will cover a systematic method for searching parameters called **grid search**, which appears not just in RC but in many analysis and optimization scenarios.
Typically, this involves multiple nested loops using `for` statements.
However, as we have already learned, `for` loops are slow in Python, and their overhead becomes significant as the number of parameters to search increases.
On the other hand, NumPy is generally more efficient as the tensor size increases, benefiting from parallelization.
Consider a method to perform grid search efficiently without using `for` loops as much as possible.
Ideally, `for` loops should only be used for the time evolution of ESN states.
Specifically, consider searching the following parameters:

- $\sigma$: input scale
- $\rho$: spectral radius
- $\mu, \kappa$: parameters of the NARMA10 model

Note that grid search for these parameters can be achieved with just a few lines of additional code, without modifying the already-implemented functions.

[ja]: #
## 参考文献

[en]: #
## References

[1] Jaeger, H. (2001). *The “echo state” approach to analysing and training recurrent neural networks-with an erratum note*. Bonn, Germany: German national research center for information technology gmd technical report, 148(34), 13.

[2] Jaeger, H. (2002). *Tutorial on training recurrent neural networks, covering BPPT, RTRL, EKF and the "echo state network" approach*. Bonn, Germany: GMD-Forschungszentrum Informationstechnik.

[3] Nakajima, K. (2020). *Physical reservoir computing—An introductory perspective*. Japanese Journal of Applied Physics, 59(6), 060501. https://doi.org/10.35848/1347-4065/ab8d4f

[4] Ceni, A., Ashwin, P., Livi, L., & Postlethwaite, C. (2020). *The echo index and multistability in input-driven recurrent neural networks.* Physica D: Nonlinear Phenomena, 412, 132609. https://doi.org/10.1016/j.physd.2020.132609

[5] Atiya, A. F., & Parlos, A. G. (2000). New results on recurrent network training: Unifying the algorithms and accelerating convergence. IEEE Transactions on Neural Networks, 11(3), 697–709. https://doi.org/10.1109/72.846741